# Imports

In [4]:
#!pip install PyPDF2
import re
import os
import pathlib
from pathlib import Path
import PyPDF2 as pdf2


# Preprocessing
## Extract PDFs to txt file

In [38]:
def _clean_line(data):
    clean_ 

def extract_pdf(pdf_file_path):
    with open (pdf_file_path, "rb") as f:
        pdf = pdf2.PdfReader(f)
        full_text = ""
        for index, page in enumerate(pdf.pages):
            # skip first page
            if index not in [0]:
                full_text += page.extract_text() + '\n'
        return full_text
    
def save_file(data: str, file_name: str):
    Path("data").mkdir(parents=True, exist_ok=True)
    file = Path("data") / file_name
    file.write_text(data)

for pdf_file in Path('data/raw').glob('*.pdf'):
    text = extract_pdf(pdf_file)
    save_file(text, pdf_file.name.split('.')[0] + '.txt')
        

07 ZETTEL.pdf
14 REMARKS ON COLOUR.pdf
09 REMARKS ON THE PHILOSOPHY OF PSYCHOLOGY 1.pdf
04 THE BLUE AND THE BROWN BOOKS.pdf
02 PHILOSOPHICAL REMARKS.pdf
(Routledge Classics) Ludwig Wittgenstein - Tractatus Logico-Philosophicus-Routledge (2001).pdf
10 REMARKS ON THE PHILOSOPHY OF PSYCHOLOGY 2.pdf
05 REMARKS ON THE FOUNDATIONS OF MATHEMATICS.pdf
08 PHILOSOPHICAL INVESTIGATIONS.pdf
01 Notebooks.pdf
06 Culture and Value.pdf
11 LAST WRITINGS ON THE PHILOSOPHY OF PSYCHOLOGY.pdf
12 LAST WRITINGS ON THE PHILOSOPHY OF PSYCHOLOGY 2.pdf
Wittgenstein - Notebooks 1914-1916 ed 1961.pdf
03 PHILOSOPHICAL GRAMMAR.pdf
13 ON CERTAINTY.pdf


In [ ]:
for txt_file in Path('data').glob('*.pdf'):
    
    break
